In [ ]:
!pip install mlxtend

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Library Import and Data Loading


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from mlxtend.frequent_patterns import apriori, association_rules


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = pd.read_csv("daraz_purchases_nepal.csv")  # Replace with your file path
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   InvoiceNo       1249 non-null   object 
 1   OrderCode       1250 non-null   int64  
 2   Description     1249 non-null   object 
 3   Quantity        1250 non-null   int64  
 4   InvoiceDate     1250 non-null   object 
 5   UnitPrice       1250 non-null   float64
 6   CustomerID      1132 non-null   float64
 7   Address         1250 non-null   object 
 8   DeliveryStatus  1250 non-null   object 
 9   Discount        1005 non-null   float64
 10  Rating          892 non-null    float64
dtypes: float64(4), int64(2), object(5)
memory usage: 107.6+ KB


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data Preprocessing


In [ ]:
df.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,OrderCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Address,DeliveryStatus,Discount,Rating
0,536365,12690,USB Type-C Cable,7,1/1/2024 0:00,147.81,51632.0,"Kathmandu, Bagmati",Pending,NaN,5.0
1,536365,84712,Portable Speaker,3,1/1/2024 1:00,67.92,85877.0,"Siddharthanagar, Lumbini",Cancelled,NaN,1.0
2,536365,29487,Power Bank,7,1/1/2024 2:00,76.36,51126.0,"Pokhara, Gandaki",Shipped,2.64,NaN
3,536365,27276,Smartwatch,3,1/1/2024 3:00,65.24,69776.0,"Pokhara, Gandaki",Cancelled,NaN,4.0
4,536365,16606,Gaming Mouse,7,1/1/2024 4:00,49.31,43802.0,"Bhaktapur, Bagmati",Delivered,9.60,4.0


In [ ]:
df.isnull().sum()


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
InvoiceNo,1
OrderCode,0
Description,1
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,118
Address,0
DeliveryStatus,0
Discount,245


In [ ]:
df_cleaned = df.dropna(subset=['CustomerID','Discount','Rating','InvoiceNo','Description'])
df_cleaned.isnull().sum()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
InvoiceNo,0
OrderCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Address,0
DeliveryStatus,0
Discount,0


In [ ]:
df_cleaned.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,OrderCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Address,DeliveryStatus,Discount,Rating
4,536365,16606,Gaming Mouse,7,1/1/2024 4:00,49.31,43802.0,"Bhaktapur, Bagmati",Delivered,9.60,4.0
7,536366,59511,Wireless Headphones,8,1/1/2024 7:00,44.32,80230.0,"Ghorahi, Lumbini",Pending,9.05,4.0
12,536367,44294,Mechanical Keyboard,6,1/1/2024 12:00,82.74,10475.0,"Damak, Koshi",Shipped,5.18,1.0
14,536367,27585,Laptop Cooling Pad,1,1/1/2024 14:00,114.04,72405.0,"Bhaktapur, Bagmati",Pending,9.81,4.0
16,536367,96121,Portable Speaker,3,1/1/2024 16:00,70.52,26618.0,"Bhaktapur, Bagmati",Pending,16.45,3.0


In [ ]:
# Remove the unnecessary spaces in the description
df_cleaned['Description'] = df_cleaned['Description'].str.strip()

df_cleaned.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
df_cleaned['InvoiceNo'] = df_cleaned['InvoiceNo'].astype('str')

In [ ]:
df_cleaned.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,OrderCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Address,DeliveryStatus,Discount,Rating
4,536365,16606,Gaming Mouse,7,1/1/2024 4:00,49.31,43802.0,"Bhaktapur, Bagmati",Delivered,9.60,4.0
7,536366,59511,Wireless Headphones,8,1/1/2024 7:00,44.32,80230.0,"Ghorahi, Lumbini",Pending,9.05,4.0
12,536367,44294,Mechanical Keyboard,6,1/1/2024 12:00,82.74,10475.0,"Damak, Koshi",Shipped,5.18,1.0
14,536367,27585,Laptop Cooling Pad,1,1/1/2024 14:00,114.04,72405.0,"Bhaktapur, Bagmati",Pending,9.81,4.0
16,536367,96121,Portable Speaker,3,1/1/2024 16:00,70.52,26618.0,"Bhaktapur, Bagmati",Pending,16.45,3.0


In [ ]:
df_cleaned['Address'].value_counts()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,count
Address,
"Pokhara, Gandaki",44
"Kathmandu, Bagmati",41
"Ghorahi, Lumbini",38
"Bhaktapur, Bagmati",37
"Janakpur, Madhesh",37
"Butwal, Lumbini",36
"Bharatpur, Lumbini",34
"Hetauda, Bagmati",34
"Tulsipur, Lumbini",34


In [ ]:
# Transactions analysis and association in Pokhara, Gandaki

CustomerOfPokhara = (df_cleaned[df_cleaned['Address'] =="Pokhara, Gandaki"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
CustomerOfPokhara.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Description,Fitness Tracker,Gaming Mouse,Laptop Cooling Pad,Mechanical Keyboard,Portable Hard Drive,Portable Speaker,Power Bank,Smartphone Cover,Smartwatch,USB Type-C Cable,VR Headset,Wireless Bluetooth Earbuds,Wireless Headphones
InvoiceNo,,,,,,,,,,,,,
536373,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
536378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
536385,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536389,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
536390,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0


# Encode the Description, i.e. items shopped together under same invoice


In [ ]:
def one_hot_encoding(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
cust_data_Pokhara_encoded = CustomerOfPokhara.map(one_hot_encoding)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
cust_data_Pokhara_encoded.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Description,Fitness Tracker,Gaming Mouse,Laptop Cooling Pad,Mechanical Keyboard,Portable Hard Drive,Portable Speaker,Power Bank,Smartphone Cover,Smartwatch,USB Type-C Cable,VR Headset,Wireless Bluetooth Earbuds,Wireless Headphones
InvoiceNo,,,,,,,,,,,,,
536373,1,0,0,0,0,0,0,0,1,0,0,0,0
536378,0,0,0,0,0,0,0,1,0,0,0,0,0
536385,0,0,0,1,0,0,0,0,0,0,0,0,0
536389,0,0,1,0,0,0,0,0,0,0,1,0,0
536390,0,0,0,0,1,0,0,0,1,0,0,0,0


In [ ]:
# Building the model
frq_items = apriori(cust_data_Pokhara_encoded, min_support = 0.01, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
rules.head(5)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
130,"(Power Bank, Laptop Cooling Pad)",(Wireless Bluetooth Earbuds),0.047619,0.047619,0.047619,1.0,21.0,1.0,0.045351,inf,1.0,1.0,1.0,1.0
131,(Wireless Bluetooth Earbuds),"(Power Bank, Laptop Cooling Pad)",0.047619,0.047619,0.047619,1.0,21.0,1.0,0.045351,inf,1.0,1.0,1.0,1.0
136,"(Smartphone Cover, Laptop Cooling Pad)",(Wireless Bluetooth Earbuds),0.047619,0.047619,0.047619,1.0,21.0,1.0,0.045351,inf,1.0,1.0,1.0,1.0
137,(Wireless Bluetooth Earbuds),"(Smartphone Cover, Laptop Cooling Pad)",0.047619,0.047619,0.047619,1.0,21.0,1.0,0.045351,inf,1.0,1.0,1.0,1.0
154,"(Power Bank, Mechanical Keyboard)",(Wireless Bluetooth Earbuds),0.047619,0.047619,0.047619,1.0,21.0,1.0,0.045351,inf,1.0,1.0,1.0,1.0


In [ ]:
# Experiment with different min_support values
for min_sup in [0.01, 0.02, 0.05]:
    frq_items = apriori(cust_data_Pokhara_encoded, min_support=min_sup, use_colnames=True)
    rules = association_rules(frq_items, metric="lift", min_threshold=1)
    print(f"Number of rules with min_support {min_sup}: {len(rules)}")

Number of rules with min_support 0.01: 414
Number of rules with min_support 0.02: 414
Number of rules with min_support 0.05: 20


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py

In [ ]:
# prompt: how can i generate the highhest frequent items in the datasets

# Assuming 'frq_items' DataFrame from the previous code is available

# Get the highest frequent items based on support
frequent_itemsets = frq_items.sort_values(by='support', ascending=False)

# Print or use the frequent_itemsets
print(frequent_itemsets.head(5)) # Print top 10 frequent itemsets


    support               itemsets
2  0.333333   (Laptop Cooling Pad)
9  0.238095  (Wireless Headphones)
3  0.238095  (Mechanical Keyboard)
7  0.238095           (Smartwatch)
8  0.238095           (VR Headset)


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
